In [0]:
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
! cp /content/drive/My\ Drive/dataset_full.zip /content/dataset_full.zip

In [0]:
!unzip dataset_full.zip

In [0]:
prot_info_initial = pd.read_csv('dataset_full (1)', sep = '\t')

In [0]:
from collections import defaultdict
zinc_id_ds_dict = defaultdict(list)   

zinc_id_db_dict = {}

In [0]:
def prot_name_by_ligand(prot_info_initial,ligand, drug_dict_local):
  #print(ligand)
  
  proteins = drug_dict_local[ligand]
  
  #print(ligand)
  #print(proteins)
  #if drug_dict_local[ligand] == []:
    #print('empty list for ', ligand)
    
  new_ds = prot_info_initial.loc[prot_info_initial['BindingDB Target Chain  Sequence'].isin(proteins)]
  prot_info = new_ds['UniProt (SwissProt) Recommended Name of Target Chain']
  prot_info=prot_info.drop_duplicates()
  proteins_for_ligand = prot_info.tolist()
  return proteins_for_ligand


def create_ds_info_dict(list_of_proteins, zinc_id_ds_dict_local):
  for prot in proteins_for_ligand:
    zinc_id_ds_dict_local[zinc_id].append(prot)
    print(zinc_id,'\t',prot,'\t',source_smile)
  return zinc_id_ds_dict_local


def create_db_info_dict(zinc_id, zinc_id_db_dict_local):
  response = get_response_by_zinc_id(zinc_id)
  soup = get_soup(response)
  list_first_col=tolist_first_column(soup)
  if len(list_first_col) != 0:
    zinc_id_db_dict_local[zinc_id]=list_first_col
  return zinc_id_db_dict_local    
      
      
      
      

In [0]:
import requests
from bs4 import BeautifulSoup


def get_response_by_zinc_id(zinc_id):
    return requests.get(rf'http://zinc.docking.org/substances/{zinc_id}/activities/').text


def get_soup(html_doc):
    return BeautifulSoup(html_doc, 'html.parser')


def stringify(content):
    if isinstance(content, str):
        return content

    return content.string


def to_strings_list(tag):
    #print(tag.contents)
    return map(stringify, tag.contents)


def prettify_string(string):
    #print(' '.join(string.split()))
    return ' '.join(string.split())


def get_pretty_string(tag):
    str_list = to_strings_list(tag)
    initial_string = ''.join(str_list)
    #print(initial_string)
    pretty_string = initial_string.replace('\n', '')
    #print(pretty_string)
    pretty_string = prettify_string(pretty_string)
    return pretty_string


def tolist_first_column(soup):
    all_tr = soup.table.tbody.find_all('tr')
    first_column = [get_pretty_string(tr.find('td')) for tr in all_tr]
    return first_column

In [0]:
from collections import defaultdict
zinc_id_ds_dict = defaultdict(list) 
zinc_id_db_dict = {}



for ind in range(5,6):
  
  
  drug_dict = defaultdict(list)
  
  with open("/content/for_cpi_prediction"+str(ind)) as file1:
    for line in file1:
      l=line.split(' ')
      #print('line', l[0])
      drug_dict[l[0]].append(l[1])
      
      
  
  print(len(drug_dict))
  for k in drug_dict:
    if drug_dict[k] == []:
      print("warning", k)
      
      
  with open("/content/lig_gen_in_zinc_ds"+str(ind)) as file2:
    first_line = file2.readline()
    unique_ids = []
    activity_data_available = []
    for line in file2:
      l=line.split(',')
      zinc_id = l[0]
      source_smile = l[3]
      if zinc_id not in unique_ids:
        unique_ids.append(zinc_id)
      proteins_for_ligand =  prot_name_by_ligand(prot_info_initial,source_smile,drug_dict)
      for prot in proteins_for_ligand:
        zinc_id_ds_dict[zinc_id].append(prot)
        #print(zinc_id,'\t',prot,'\t',source_smile)
      response = get_response_by_zinc_id(zinc_id)
      soup = get_soup(response)
      list_first_col=tolist_first_column(soup)
      if len(list_first_col) != 0:
        zinc_id_db_dict[zinc_id]=list_first_col
        if zinc_id not in activity_data_available:
          activity_data_available.append(zinc_id)
    print('number of ligands in zinc db ', len(unique_ids))  
    print('activity data available ', len(activity_data_available))
        
        
    print(len(drug_dict))

In [0]:
out=open("/content/in_zinc_results", "w")
for zinc_id in zinc_id_db_dict:
  #if zinc_id in zinc_id_ds_dict:
  out.write(zinc_id+'\t'+str(zinc_id_ds_dict[zinc_id])+'\t'+str(zinc_id_db_dict[zinc_id])+'\n')
    
out.close()

In [0]:
print('dict length ds ',len(zinc_id_ds_dict),'dict length db ',len(zinc_id_db_dict))

In [0]:
for i in unique_ids:
  if i not in zinc_id_ds_dict:
    print(i)

In [0]:
for i in activity_data_available:
  if i not in zinc_id_db_dict:
    print(i)

In [0]:
!cp /content/drive/My\ Drive/BindingDB_All_2019m3.tsv.zip /content/

In [0]:
!unzip BindingDB_All_2019m3.tsv.zip

In [0]:
data = pd.read_csv('BindingDB_All.tsv', sep = '\t', error_bad_lines=False)

In [0]:
new_ds = prot_info_initial.loc[prot_info_initial['BindingDB Target Chain  Sequence'].isin(['MAAASSPPRAERKRWGWGRLPGARRGSAGLAKKCPFSLELAEGGPAGGALYAPIAPGAPGPAPPASPAAPAAPPVASDLGPRPPVSLDPRVSIYSTRRPVLARTHVQGRVYNFLERPTGWKCFVYHFAVFLIVLVCLIFSVLSTIEQYAALATGTLFWMEIVLVVFFGTEYVVRLWSAGCRSKYVGLWGRLRFARKPISIIDLIVVVASMVVLCVGSKGQVFATSAIRGIRFLQILRMLHVDRQGGTWRLLGSVVFIHRQELITTLYIGFLGLIFSSYFVYLAEKDAVNESGRVEFGSYADALWWGVVTVTTIGYGDKVPQTWVGKTIASCFSVFAISFFALPAGILGSGFALKVQQKQRQKHFNRQIPAAASLIQTAWRCYAAENPDSSTWKIYIRKAPRSHTLLSPSPKPKKSVVVKKKKFKLDKDNGVTPGEKMLTVPHITCDPPEERRLDHFSVDGYDSSVRKSPTLLEVSMPHFMRTNSFAEDLDLEGETLLTPITHISQLREHHRATIKVIRRMQYFVAKKKFQQARKPYDVRDVIEQYSQGHLNLMVRIKELQRRLDQSIGKPSLFISVSEKSKDRGSNTIGARLNRVEDKVTQLDQRLALITDMLHQLLSLHGGSTPGSGGPPREGGAHITQPCGSGGSVDPELFLPSNTLPTYEQLTVPRRGPDEGS'])]

In [0]:
new_ds.shape

In [0]:
for ind in range(1,6):
  print(ind)
  with open("/content/lig_gen_in_zinc_1000_ds"+str(ind)) as file2:
      first_line = file2.readline()
      unique_ids = []
      for line in file2:
        l=line.split(',')
        zinc_id = l[0]
        if zinc_id not in unique_ids:
          unique_ids.append(zinc_id)      
      print('number of ligands in zinc db ', len(unique_ids))
        
        
